![](assets/mpcs56420.jpeg)

# BLASTP Search¶

In [117]:
# Download a database from NCBI
#!wget https://ftp.ncbi.nlm.nih.gov/blast/db/FASTA/pdbaa.gz -P downloads/
# Unzip the database
#!gunzip downloads/pdbaa.gz
#!mv downloads/pdbaa downloads/pdbaa.fasta

#!wget https://ftp.ncbi.nlm.nih.gov/blast/db/FASTA/swissprot.gz -P downloads/



--2020-10-21 23:10:30--  https://ftp.ncbi.nlm.nih.gov/blast/db/FASTA/swissprot.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 2607:f220:41e:250::11, 2607:f220:41e:250::10, 2607:f220:41e:250::7, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|2607:f220:41e:250::11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 139788715 (133M) [application/x-gzip]
Saving to: ‘downloads/swissprot.gz’

swissprot.gz        100%[===================>] 133.31M  1022KB/s    in 3m 14s  

2020-10-21 23:13:44 (704 KB/s) - ‘downloads/swissprot.gz’ saved [139788715/139788715]



## Create a blast compatible db

In [86]:
!makeblastdb -in downloads/pdbaa.fasta -input_type fasta -dbtype prot -out db/pdbaa



Building a new DB, current time: 10/21/2020 15:39:39
New DB name:   /Users/tabinkowski/Desktop/BLAST/assignment4/db/pdbaa
New DB title:  downloads/pdbaa.fasta
Sequence type: Protein
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 119997 sequences in 5.78093 seconds.


## Run BLASTP

In [121]:
# Run a blast job
#!blastp -query assets/ndm1.fasta -db db/pdbaa -outfmt 6 -out protein1.out
!blastp -query assets/ndm1.fasta -db db/pdbnt -outfmt 6 -out protein6.out

BLAST Database error: No alias or index file found for protein database [db/pdbnt] in search path [/Users/tabinkowski/Desktop/BLAST/assignment4::]


In [109]:
# Check the output of the tabular results with the UNIX "head" command just to 
# make sure something is there
!head protein1.out

WP_004201164.1	4EXS_A	100.000	270	0	0	1	270	1	270	0.0	556
WP_004201164.1	4EY2_A	100.000	270	0	0	1	270	1	270	0.0	555
WP_004201164.1	3SPU_A	100.000	245	0	0	27	271	1	245	0.0	505
WP_004201164.1	4U4L_A	99.593	246	1	0	25	270	1	246	0.0	505
WP_004201164.1	6IBS_A	100.000	244	0	0	27	270	1	244	0.0	504
WP_004201164.1	5N0I_A	100.000	243	0	0	29	271	1	243	0.0	501
WP_004201164.1	5N0H_A	100.000	243	0	0	29	271	1	243	0.0	501
WP_004201164.1	6JKB_A	100.000	243	0	0	28	270	2	244	0.0	501
WP_004201164.1	5XP6_A	100.000	242	0	0	29	270	2	243	0.0	499
WP_004201164.1	4RL2_A	98.780	246	3	0	25	270	1	246	0.0	499


In [111]:
# Run a blast job with more output options for output format 6 (https://github.com/seqan/lambda/wiki/BLAST-Output-Formats)
!blastp -query assets/ndm1.fasta -db db/pdbaa -outfmt "6 qseqid sseqid stitle qstart qend sstart send length pident bitscore evalue" -out protein1.out -evalue 10

# Sanity check
#!head protein1.out

In [116]:
import pandas as pd

# Load blast results into dataframe and rename columns with meaningful values
blast_results = pd.read_csv('protein1.out',
                           '\t',
                           header=None).rename(columns={0:'qseqid',
                                                        1:'sseqid',
                                                        2:'stitle',
                                                        3:'qstart',
                                                        4:'qend',
                                                        5:'sstart',
                                                        6:'ssend',
                                                        7:'length',
                                                        8:'pident',
                                                        9:'bitscore',
                                                        10:'evalue',
                                                       })

# Reformat sseqid values so they are usuable for downstream operations
blast_results.stitle = [x.split('_')[1] for x in blast_results.stitle]
blast_results

,qseqid,sseqid,stitle,qstart,qend,sstart,ssend,length,pident,bitscore,evalue
0,WP_004201164.1,4EXS_A,"A Chain A, Beta-lactamase Ndm-1 [Klebsiella pn...",1,270,1,270,270,100.000,556.0,0.0
1,WP_004201164.1,4EY2_A,"A Chain A, Beta-lactamase NDM-1 [Klebsiella pn...",1,270,1,270,270,100.000,555.0,0.0
2,WP_004201164.1,3SPU_A,"A Chain A, Beta-lactamase NDM-1 [Klebsiella pn...",27,271,1,245,245,100.000,505.0,0.0
3,WP_004201164.1,4U4L_A,"A Chain A, Beta-lactamase Ndm-1 [Klebsiella pn...",25,270,1,246,246,99.593,505.0,0.0
4,WP_004201164.1,6IBS_A,"A Chain A, Metallo-beta-lactamase type 2 [Kleb...",27,270,1,244,244,100.000,504.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
154,WP_004201164.1,2P4Z_A,"A Chain A, Metal-dependent Hydrolases Of The B...",73,153,29,113,92,28.261,31.2,1.3
155,WP_004201164.1,3Q5Z_A,"A Chain A, Rop5b [Toxoplasma gondii] 3Q60",126,208,235,322,88,27.273,31.6,1.3
156,WP_004201164.1,6JL3_A,"A Chain A, Ubiquitin domain-containing protein...",191,235,35,79,45,26.667,28.5,3.1
157,WP_004201164.1,4LV8_A,"A Chain A, Rhoptry protein 5 C [Toxoplasma gon...",126,208,235,322,88,25.000,29.6,5.5


In [115]:
# We can use the dataframe to extract other information. In the next two cells, 
# we'll check how many rows are in the table, and how many unique database matches
# we found.

print('There are {} alignments, with {} unique subject sequences'.format(blast_results.index.size,
                                                                         blast_results.sseqid.unique().size))

# generate descriptive statistics for numerical columns
blast_results.describe()

There are 159 alignments, with 159 unique subject sequences


,qstart,qend,sstart,ssend,length,pident,bitscore,evalue
count,159.000000,159.000000,159.000000,159.000000,159.000000,159.000000,159.000000,1.590000e+02
mean,45.427673,261.301887,15.081761,226.874214,219.446541,52.901881,232.240881,1.099948e-01
std,21.688374,18.968914,27.320176,27.285120,31.848861,30.644030,170.463741,6.839420e-01
min,1.000000,132.000000,1.000000,79.000000,45.000000,19.307000,28.500000,0.000000e+00
25%,31.000000,265.000000,4.000000,215.000000,215.000000,32.863500,122.000000,1.053500e-169
50%,47.000000,267.000000,10.000000,230.000000,221.000000,34.389000,151.000000,3.750000e-44
75%,48.000000,270.000000,19.000000,243.000000,239.000000,99.136000,469.000000,2.835000e-33
max,191.000000,271.000000,235.000000,322.000000,270.000000,100.000000,556.000000,5.700000e+00


## Cleanup 
We see that there are multiple sequences with the same title. This may represent duplicate entries in a database or high-scoring subsequences matches.

In [106]:
# Remove duplicats based on the name
# This may not provide the results you are looking for
blast_results[blast_results.duplicated('stitle', False)]

,qseqid,sseqid,stitle,qstart,qend,sstart,ssend,length,pident,bitscore,evalue
42,YP_009724390.1,6XDG_E,"E Chain E, Spike glycoprotein [Severe acute re...",319,541,1,223,223,100.000,470.0,7.270000e-157
56,YP_009724390.1,6ZCZ_E,"E Chain E, Spike glycoprotein [Severe acute re...",333,528,10,205,196,99.490,411.0,3.700000e-135
61,YP_009724390.1,6VSJ_A,"A Chain A, Spike glycoprotein [Murine hepatiti...",530,1189,579,1249,696,36.494,404.0,5.830000e-120
62,YP_009724390.1,6VSJ_A,"A Chain A, Spike glycoprotein [Murine hepatiti...",259,436,256,436,183,30.601,85.1,4.750000e-16
63,YP_009724390.1,3JCL_A,"A Chain A, Spike glycoprotein [Murine hepatiti...",530,1195,558,1230,701,36.091,402.0,4.490000e-119
64,YP_009724390.1,3JCL_A,"A Chain A, Spike glycoprotein [Murine hepatiti...",259,436,235,415,183,30.601,85.5,4.030000e-16
65,YP_009724390.1,6NZK_A,"A Chain A, Spike surface glycoprotein [Human c...",528,1195,621,1287,691,36.035,401.0,1.420000e-118
66,YP_009724390.1,6NZK_A,"A Chain A, Spike surface glycoprotein [Human c...",242,437,254,446,197,30.457,98.6,3.580000e-20
67,YP_009724390.1,5I08_A,"A Chain A, Spike glycoprotein,Foldon chimera [...",530,1196,598,1265,695,35.252,398.0,1.390000e-117
68,YP_009724390.1,5I08_A,"A Chain A, Spike glycoprotein,Foldon chimera [...",265,436,243,414,173,30.636,92.8,2.360000e-18
